In [3]:
import numpy as np
import pandas as pd
import math
import yfinance as yf

In [7]:
import os
os.getcwd()
all_files = os.listdir('.')
print("Hidden files:", [f for f in all_files if f.startswith('.')])
os.chdir('/Volumes/Macintosh HD 2 - Data/Users/vikrambaha/QF-YEAR-2/4thSemester/EMP/project')          # Change directory if needed

no_tikcer_data=pd.read_csv("data/no_ticker_data_found.csv")
groups=no_tikcer_data.groupby("country")
groups.count()

Hidden files: ['.DS_Store', '.ipynb_checkpoints']


,risk_rating_score,risk_rating_assessment,industry_group,identifier,company_name,ticker,return,volatility
country,,,,,,,,
Argentina,4,4,4,4,4,4,0,0
Armenia,1,1,1,1,1,1,0,0
Australia,57,57,57,57,57,57,0,0
Austria,16,16,16,16,16,16,0,0
Bahamas,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...
Uzbekistan,3,3,3,3,3,3,0,0
Venezuela,1,1,1,1,1,1,0,0
Vietnam,15,15,15,15,15,15,0,0


In [104]:
groups.country.value_counts().nlargest(20)
df_hk = groups.get_group("India")
df_hk.to_csv("data/India_identifiers.csv",index=False)

In [106]:
# df_hk = pd.read_csv("data/hk_identifiers.csv")
df_hk.head()

,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,company_name,ticker,return,volatility
2,24.7,Medium Risk,Diversified Financials,BOM:542772,India,360-one-wam-ltd,542772.BO,NaN,NaN
3,22.9,Medium Risk,Industrial Conglomerates,BOM:523395,India,3m-india-ltd,523395.BO,NaN,NaN
10,28.6,Medium Risk,Diversified Financials,BOM:544176,India,aadhar-housing-finance-ltd,544176.BO,NaN,NaN
13,32.8,High Risk,Pharmaceuticals,BOM:524348,India,aarti-drugs-ltd,524348.BO,NaN,NaN
14,28.3,Medium Risk,Chemicals,BOM:524208,India,aarti-industries-ltd,524208.BO,NaN,NaN


In [91]:
def calculate_yearly_returns_volatility(ticker):
    try:
        if (len(ticker)>3):
            ticker = ticker.replace(".J",".JO")
        # ticker=format_hk_string(ticker)
        data = yf.download(ticker, start='2024-01-01', end='2024-12-31')
        if data.empty:
            return None

        # Calculate daily returns
        data['Daily Return'] = data['Close'].pct_change()

        # Calculate yearly return
        yearly_return = (1 + data['Daily Return']).prod() - 1

        # Calculate yearly volatility (annualized)
        yearly_volatility = data['Daily Return'].std() * (252**0.5)

        return (round(float(yearly_return),4), round(float(yearly_volatility),4))
    except Exception as e:
        print(f"An error occurred: {e}")
        return (-99, -99)

In [93]:
results = {}

for indx, r in df_hk.iterrows():
    yearly_data = calculate_yearly_returns_volatility(r['ticker'])
    results[r['ticker']] = yearly_data

# # Create a DataFrame from the results dictionary


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['-']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[******************

In [109]:
# del results['-']
# results
# df_results = pd.DataFrame.from_dict(results, orient='index', columns=['return', 'volatility'])
df_results.head()
# df_results = pd.read_csv('data/India_return_vol_short.csv')
# df_results.head()
del df_hk['return']
del df_hk['volatility']

In [110]:
# df_results=df_results.reset_index()
# df_results = df_results.rename(columns={'index': 'ticker'})
df_joined_merge = pd.merge(df_hk, df_results, on='ticker', how='left')
df_joined_merge.head()

,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,company_name,ticker,return,volatility
0,24.7,Medium Risk,Diversified Financials,BOM:542772,India,360-one-wam-ltd,542772.BO,0.7780,0.3868
1,22.9,Medium Risk,Industrial Conglomerates,BOM:523395,India,3m-india-ltd,523395.BO,-0.1699,0.2696
2,28.6,Medium Risk,Diversified Financials,BOM:544176,India,aadhar-housing-finance-ltd,544176.BO,0.2825,0.4176
3,32.8,High Risk,Pharmaceuticals,BOM:524348,India,aarti-drugs-ltd,524348.BO,-0.0697,0.2963
4,28.3,Medium Risk,Chemicals,BOM:524208,India,aarti-industries-ltd,524208.BO,-0.3708,0.3871


In [111]:
df_joined_merge.to_csv("data/India_return_vol.csv", index=False)

In [60]:
def format_hk_string(hk_string):
    try:
        # Split the string into the digit part and the ".HK" part
        parts = hk_string.split(".")
        if len(parts) != 2 or parts[1] != "HK":
            return hk_string  # Return original if not in expected format

        digit_part = parts[0]

        # Convert the digit part to an integer
        digit_int = int(digit_part)

        # Format the integer to four digits with leading zeros
        formatted_digit_part = "{:04d}".format(digit_int)

        # Combine the formatted digit part and ".HK"
        formatted_hk_string = formatted_digit_part + ".HK"
        return formatted_hk_string

    except ValueError:
        return hk_string  # Return original if the digit part is not a valid integer
    except IndexError:
        return hk_string # Return original if the string does not contain '.'



In [113]:
df1=pd.read_csv("data/China_return_vol.csv")
df2=pd.read_csv("data/hk_return_vol.csv")
df3=pd.read_csv("data/India_return_vol.csv")
df4=pd.read_csv("data/SA_return_vol.csv")
df5=pd.read_csv("data/sweden_return_vol.csv")
df6=pd.read_csv("data/Turkey_return_vol.csv")


In [114]:
main_df=pd.concat([df1,df2,df3,df4,df5,df6], ignore_index=True)

In [118]:
esg_df = pd.read_csv("data/esg_return_volatility.csv")

In [120]:
df_joined_merge = pd.merge(esg_df, main_df, on='ticker', how='left')

In [127]:
esg_df[esg_df['ticker']=='753.HK']

,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,company_name,ticker,return,volatility
348,28.7,Medium Risk,Transportation,HKG:753,China,air-china-ltd,753.HK,NaN,NaN


In [126]:
main_df[main_df['ticker']=='753.HK']

,risk_rating_score,risk_rating_assessment,industry_group,identifier,country,company_name,ticker,return,volatility
1,28.7,Medium Risk,Transportation,HKG:753,China,air-china-ltd,753.HK,0.0606,0.4685


In [132]:
def merge_dataframes_fill_nan(df1, df2, on_column):
    try:
        # Create a copy to avoid modifying the original DataFrames
        df1_copy = df1.copy()

        # Iterate through the rows of the first dataframe
        for index, row in df1_copy.iterrows():
            # Get the value of the on_column for the current row
            key_value = row[on_column]

            # Check if this value exists in the second dataframe's on_column
            matching_rows_df2 = df2[df2[on_column] == key_value]

            if not matching_rows_df2.empty:
                df1_copy.loc[index, 'return'] = matching_rows_df2.iloc[0]['return']
                df1_copy.loc[index, 'volatility'] = matching_rows_df2.iloc[0]['volatility']
                # If a match is found, update the first dataframe with values from the second dataframe
                # for col in df1_copy.columns:
                #     if col != on_column:
                #         # Assuming that df2 has only one matching row for each key_value in df1
                #         df1_copy.loc[index, col] = matching_rows_df2.iloc[0][col]

        return df1_copy

    except KeyError as e:
        print(f"Error: Column '{on_column}' not found in one or both dataframes.  Ensure the column name is correct.")
        print(e)
        return pd.DataFrame()  # Return an empty DataFrame on error
    except Exception as e:
        print(f"An unexpected error occurred during the merge and fill operation: {e}")
        return pd.DataFrame()  # Return empty DataFrame



In [133]:
result_df = merge_dataframes_fill_nan(esg_df, main_df, on_column='ticker')


In [154]:
# result_df[result_df['volatility']==-99.0]
# result_df = result_df.dropna(subset=['return'])
len(result_df)

11429

In [153]:
# result_df[result_df['return'].isnull()]
result_df.to_csv("data/esg_return_vol_clean_data.csv", index=False)